<a href="https://colab.research.google.com/github/sandhyaparna/Python-DataScience-CookBook/blob/master/Natural%20Language%20Processing/NLP_in_Tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow==2.3.2
!pip install tensorflow-text==2.3.0  # tensorflow_text
!pip install tfx==0.26.1 
!pip install tf-models-official==2.3.0

* Use clothing data set to develop a pre-trained model 
  * Amazon product descriptions in metadata files: http://deepyeti.ucsd.edu/jianmo/amazon/index.html
  * Read data from json files: http://jmcauley.ucsd.edu/data/amazon/ ; https://nijianmo.github.io/amazon/index.html

* clothing/fashion word embeddings https://making.lyst.com/2014/11/11/word-embeddings-for-fashion/
* https://github.com/mvasil/fashion-compatibility
* general product descriptions data: https://data.world/promptcloud/fashion-products-on-amazon-com/workspace/file?filename=amazon_co-ecommerce_sample.csv

In [4]:
import matplotlib.pyplot as plt
import os
import re
import shutil
import string
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras import preprocessing
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

* https://www.rexegg.com/regex-quickstart.html
* https://docs.python.org/3/library/re.html
* http://www.pyregex.com/
* https://www.dataquest.io/blog/regular-expressions-data-scientists/
* All Unicode characters https://en.wikipedia.org/wiki/List_of_Unicode_characters

### Cleaning functions
 

In [ ]:
example_text = ""

tf.strings.lower(example_text)  # converts example_text to lowercase

tf.strings.regex_replace(example_text, '<br />', '') # Replaces line breaks with emptystring in example_text 

# string.punctuation contains !"#$%&'()*+, -./:;<=>?@[\]^_`{|}~   # special characters like these are not removed
# similar to string.punctuation functions https://docs.python.org/3/library/string.html
# re.escape is to escape special characters in above
# % is used to point to string.punctuation
# Any punctuation attached to present seperately in the string is removed
string.punctuation+'®' # to add more characters to string.punctuation
tf.strings.regex_replace(example_text,'[%s]' % re.escape(string.punctuation),'') # Replaces line breaks with emptystring in example_text 

# To convert/encode unicode characters 
example_text.encode("utf-8")
# To decode the format back to unicode character
example_text.dencode("utf-8")

# Extract only text data without labels
raw_train_ds.map(lambda x, y: x)

# To get vocab size of the vectorize layer
print('Vocabulary size: {}'.format(len(vectorize_layer.get_vocabulary())))

# Identify all unicode characters
[^\u0000-\u007e]+

# Retains only numbers or letters
Df['Text_col'].apply(lambda x: re.findall("\w+",str(x)))



In [11]:
import tensorflow as tf
import tensorflow_hub as hub

hub_url = "https://tfhub.dev/google/nnlm-en-dim128/2"
embed = hub.KerasLayer(hub_url)
embeddings = embed(["A long sentence.", "single-word", "http://example.com"])
print(embeddings.shape, embeddings.dtype)

(3, 128) <dtype: 'float32'>


### BERT
* Difference between various BERT pre-trained models: https://tfhub.dev/google/collections/bert/1
* L corresponds to number of hidden layers
* H is hidden size
* A is Attention heads

In [ ]:
import os
import shutil

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
# from official.nlp import optimization  # to create AdamW optmizer

# import matplotlib.pyplot as plt

tf.get_logger().setLevel('ERROR')

In [11]:
# BERT pre-process model
tfhub_handle_preprocess = 'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3'
bert_preprocess_model = hub.KerasLayer(tfhub_handle_preprocess)

# BERT pre-trained model
tfhub_handle_encoder = 'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1'
bert_model = hub.KerasLayer(tfhub_handle_encoder)

text_test = ['this is such an amazing movie']
text_preprocessed = bert_preprocess_model(text_test)

# input_word_ids contains numerical ids for each tokenized input
print(f'Keys       : {list(text_preprocessed.keys())}')

# Max sequence length for any BERT model = 512
print(f'Shape      : {text_preprocessed["input_word_ids"].shape}')  # sequence length = 128

# Inputs are tokenized using vocab file
print(f'Word Ids   : {text_preprocessed["input_word_ids"][0, :12]}')  # Tokenized sequence

# 1 for useful tokens, 0 for padding
print(f'Input Mask : {text_preprocessed["input_mask"][0, :12]}')  # Binary encoding to differentiate padded tokens

print(f'Type Ids   : {text_preprocessed["input_type_ids"][0, :12]}')  # indicates whether the token i belongs to sentence 1 (value 0) or 2 (value 1)

bert_results = bert_model(text_preprocessed)

# outputs=[pooled_output, sequence_output])

# sequence_output for 1 sentence is 512 dimensions for 128 tokens (as 128 is max sequence length)

# Each position outputs a vector of 512. Outputs classification CLS token
# We are interested in only CLS token to pass through further keras layer for classification/regression.
# pooled_output shape of CLS token is [batch_size, 512]  


print(f'Loaded BERT: {tfhub_handle_encoder}')
print(f'Pooled Outputs Shape:{bert_results["pooled_output"].shape}')
print(f'Pooled Outputs Values:{bert_results["pooled_output"][0, :12]}')
print(f'Sequence Outputs Shape:{bert_results["sequence_output"].shape}')
print(f'Sequence Outputs Values:{bert_results["sequence_output"][0, :12]}')

Keys       : ['input_word_ids', 'input_mask', 'input_type_ids']
Shape      : (1, 128)
Word Ids   : [ 101 2023 2003 2107 2019 6429 3185  102    0    0    0    0]
Input Mask : [1 1 1 1 1 1 1 1 0 0 0 0]
Type Ids   : [0 0 0 0 0 0 0 0 0 0 0 0]
Loaded BERT: https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1
Pooled Outputs Shape:(1, 512)
Pooled Outputs Values:[ 0.94622993  0.96978396  0.13774446  0.27936164 -0.3334526   0.34829125
  0.94912225 -0.9515633  -0.00178645 -0.93107945 -0.04124452 -0.97975695]
Sequence Outputs Shape:(1, 128, 512)
Sequence Outputs Values:[[-0.34628797  0.25210583  0.5756581  ... -0.15549679  0.472016
   0.37473977]
 [-0.5864688   0.31059366 -0.27019346 ...  0.5594992  -0.23402517
   0.92162156]
 [-0.87764287  0.43894887 -0.61759305 ...  0.0450628  -0.34966043
   0.43843132]
 ...
 [-0.3622594  -0.2679375  -0.04834296 ...  0.40202618  0.5345558
   0.5671515 ]
 [-1.0682071  -0.670603    0.14062242 ...  0.12432958  0.40765905
   0.7977506 ]
 [-0.4245

In [4]:
import tensorflow_text as text

tfhub_handle_preprocess = 'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3'
bert_preprocess_model = hub.KerasLayer(tfhub_handle_preprocess)

# BERT pre-trained model
tfhub_handle_encoder = 'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1'
bert_model = hub.KerasLayer(tfhub_handle_encoder)

from sklearn.metrics.pairwise import cosine_similarity

feature_vec_1 = bert_model(bert_preprocess_model(["3/4 sleeve"]))["pooled_output"]
feature_vec_2 = bert_model(bert_preprocess_model(["3 4 sleeve"]))["pooled_output"]

cosine_similarity(feature_vec_1, feature_vec_2)

In [49]:
import tensorflow_text as text

tfhub_handle_preprocess = 'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3'
bert_preprocess_model = hub.KerasLayer(tfhub_handle_preprocess)

# BERT pre-trained model
tfhub_handle_encoder = 'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1'
bert_model = hub.KerasLayer(tfhub_handle_encoder)

from sklearn.metrics.pairwise import cosine_similarity

def cosine_similarity_fn(text1,text2):
  feature_vec_1 = bert_model(bert_preprocess_model([text1]))["pooled_output"]
  feature_vec_2 = bert_model(bert_preprocess_model([text2]))["pooled_output"]
  return 'similarity of' , text1,'and', text2, cosine_similarity(feature_vec_1, feature_vec_2)

print(cosine_similarity_fn("3/4 sleeve","3 4 sleeve"))

('similarity of', '3/4 sleeve', 'and', '3 4 sleeve', array([[0.9320507]], dtype=float32))


In [ ]:
# {' ',
#  '!',
#  '"',
#  '#',
#  '$',
#  '%',
#  '&',
#  "'",
#  '(',
#  ')',
#  '*',
#  '+',
#  ',',
#  '-',
#  '.',
#  '/',
#  ':',
#  ';',
#  '<',
#  '=',
#  '>',
#  '?',
#  '_'}

In [ ]:
# "!" - can be left as is
# '"' - eg: 4" seem; 
# ? needs to be removed



In [48]:
raw_text = r'I have a 15 " laptop, 4 " seam "tide plus"'
re.findall(r'(.....)"(.......)',raw_text)
# re.findall(r'AV', 'AV Analytics Vidhya AV')

[('a 15 ', ' laptop'), ('seam ', 'tide pl')]

In [10]:
print(cosine_similarity_fn("don't","do not"))
print(cosine_similarity_fn("they're","they are"))
print(cosine_similarity_fn("it's","it is"))
print(cosine_similarity_fn("it?s","it is"))
print(cosine_similarity_fn("easy-to-layer","easy to layer"))
print(cosine_similarity_fn("easy-to-layer","easy to layer"))


('similarity of', "don't", 'and', 'do not', array([[0.9083276]], dtype=float32))
('similarity of', "they're", 'and', 'they are', array([[0.8668688]], dtype=float32))
('similarity of', "it's", 'and', 'it is', array([[0.76479155]], dtype=float32))
('similarity of', 'it?s', 'and', 'it is', array([[0.65618396]], dtype=float32))
('similarity of', 'easy-to-layer', 'and', 'easy to layer', array([[0.89126873]], dtype=float32))


In [5]:
def cosine_similarity_fn(text1,text2):
  feature_vec_1 = bert_model(bert_preprocess_model([text1]))["pooled_output"]
  feature_vec_2 = bert_model(bert_preprocess_model([text2]))["pooled_output"]
  return 'similarity of' , text1,'and', text2, cosine_similarity(feature_vec_1, feature_vec_2)
  # return "similarity of %s and %s" % (text1,text2), text1, " and ", text2, cosine_similarity(feature_vec_1, feature_vec_2)
# cosine_similarity_fn("3/4 sleeve","3 4 sleeve")

In [49]:
def preprocessing_fn(input_data):
  lowercase = tf.strings.lower(input_data)
  linebreaks = tf.strings.regex_replace(lowercase, '<br />', ' ')
  stripped_html = tf.strings.regex_replace(linebreaks,
                                  '[%s]' % re.escape(string.punctuation),'')
  return tf.strings.regex_replace(stripped_html, '[^\u0000-\u007e]+', '')

In [54]:
print(cosine_similarity_fn(preprocessing_fn(" Standard size 1.7 fl oz/ 30 ml.")," Standard size 1.7 fl oz/ 30 ml."))
print(cosine_similarity_fn(" Standard size 1.7 fl oz  30 ml."," Standard size 1.7 fl oz/ 30 ml."))
print(cosine_similarity_fn("don't","do not"))


('similarity of ', <tf.Tensor: shape=(), dtype=string, numpy=b' standard size 17 fl oz 30 ml'>, ' and ', ' Standard size 1.7 fl oz/ 30 ml.', array([[0.6451631]], dtype=float32))
('similarity of ', ' Standard size 1.7 fl oz  30 ml.', ' and ', ' Standard size 1.7 fl oz/ 30 ml.', array([[0.9953242]], dtype=float32))
('similarity of ', "don't", ' and ', 'do not', array([[0.9083276]], dtype=float32))


In [47]:
print(cosine_similarity_fn("3/4 sleeve","3 4 sleeve"))
print(cosine_similarity_fn("3/4 sleeve","3 by 4 sleeve"))


print(cosine_similarity_fn("3 ®","3 \xc2\xae"))
print(cosine_similarity_fn("3®","3\xc2\xae"))
print(cosine_similarity_fn("3®","3"))
print(cosine_similarity_fn("3 ®","3®"))
print(cosine_similarity_fn("A/B","A by B"))


('similarity of ', '3/4 sleeve', ' and ', '3 4 sleeve', array([[0.9320507]], dtype=float32))
('similarity of ', '3/4 sleeve', ' and ', '3 by 4 sleeve', array([[0.96390617]], dtype=float32))
('similarity of ', '3 ®', ' and ', '3 Â®', array([[0.87759125]], dtype=float32))
('similarity of ', '3®', ' and ', '3Â®', array([[0.9169636]], dtype=float32))
('similarity of ', '3®', ' and ', '3', array([[0.8828571]], dtype=float32))
('similarity of ', '3 ®', ' and ', '3®', array([[0.8762013]], dtype=float32))
('similarity of ', 'A/B', ' and ', 'A by B', array([[0.8666401]], dtype=float32))


In [45]:
print("®".encode())

b'\xc2\xae'


In [8]:
string_check = "� � 3® ° 200gsm (Bright Lycra)"
string_check.encode("utf-8")

b'\xef\xbf\xbd \xef\xbf\xbd 3\xc2\xae \xc2\xb0 200gsm (Bright Lycra)'

In [ ]:
text_test = ['®!#$%&()*+,-./:;<=>?@[\]^_`{|}~®'] 
# text_test = ['\xef\xbf\xbd'] #�

### Activation functions for Regression(y>0) :
* softplus
* ReLU
* Exponential
* Piece-wise linear

### loss functions
* Region Proposal Network box
* object Localization Network RPN centerness 

### https://www.tensorflow.org/tutorials/keras/text_classification
* In Text Vectorize layer: Each word is given a unique number and encoding of the sentence of the sentence is based on that. Transforms strings into vocab indices
* This Vectorize layer is passed through the keras embedding layer to get embedding vectors for those words. Embedding vector is learnt during training


In [ ]:
### Tensorflow Notebooks
""" Data is extracted from url into folders. Then data is passed through 'tf.keras.preprocessing.text_dataset_from_directory' function to 
prepare data into suitable format """

# Function for a sneak peek into the data </br>
for text_batch, label_batch in raw_train_ds.take(1): 
  for i in range(3): 
    print("Review", text_batch.numpy()[i]) 
    print("Label", label_batch.numpy()[i])  

In [ ]:
def custom_standardization(input_data):
  lowercase = tf.strings.lower(input_data)
  stripped_html = tf.strings.regex_replace(lowercase, '<br />', '')
  return tf.strings.regex_replace(stripped_html,
                                  '[%s]' % re.escape(string.punctuation),
                                  '')

# Added removal of unicode characters to the above code  
def custom_standardization_check(input_data):
  lowercase = tf.strings.lower(input_data)
  linebreaks = tf.strings.regex_replace(lowercase, '<br />', ' ')
  stripped_html = tf.strings.regex_replace(linebreaks,
                                  '[%s]' % re.escape(string.punctuation),' ')
  return tf.strings.regex_replace(stripped_html, '[^\u0000-\u007e]+', ' ')

raw_data_sample = 'ത'+'"Pandemonium" -is a horror® movie spoof - !that comes off more stupid than funny.<br /><br />Believe me when I tell you, I love comedies. Especially comedy spoofs. "Airplane", "The Naked Gun" trilogy, "Blazing Saddles", "High Anxiety", and "Spaceballs" are some of my favorite comedies that spoof a particular genre. "Pandemonium" is not up there with those films. Most of the scenes in this movie had me sitting there in stunned silence because the movie wasn\'t all that funny. There are a few laughs in the film, but when you watch a comedy, you expect to laugh a lot more than a few times and that\'s all this film has going for it. Geez, "Scream" had more laughs than this film and that was more of a horror film. How bizarre is that?<br /><br />*1/2 (out of four)'
custom_standardization(raw_data_sample)

<tf.Tensor: shape=(), dtype=string, numpy=b'\xe0\xb4\xa4pandemonium is a horror\xc2\xae movie spoof  that comes off more stupid than funnybelieve me when i tell you i love comedies especially comedy spoofs airplane the naked gun trilogy blazing saddles high anxiety and spaceballs are some of my favorite comedies that spoof a particular genre pandemonium is not up there with those films most of the scenes in this movie had me sitting there in stunned silence because the movie wasnt all that funny there are a few laughs in the film but when you watch a comedy you expect to laugh a lot more than a few times and thats all this film has going for it geez scream had more laughs than this film and that was more of a horror film how bizarre is that12 out of four'>

In [ ]:
# ADAPT the vectorize_layer on only Text part of train dat (remove label)
vectorize_layer = TextVectorization(
    standardize=custom_standardization,  # Default is 'lower_and_strip_punctuation'.
    max_tokens=None,  # no cap on number of tokens
    output_mode='int',
    output_sequence_length=sequence_length)

# Make a text-only dataset (without labels), then call adapt
def vectorize_text(text, label):
  text = tf.expand_dims(text, -1)
  return vectorize_layer(text), label

In [ ]:
# To retrieve a batch (of 32 reviews and labels) from the dataset: loop is added to the existing code
text_batch, label_batch = next(iter(raw_train_ds))
for i in range(len(text_batch)):
  first_review, first_label = text_batch[i], label_batch[i]
  print("Review", first_review)
  print("Label", raw_train_ds.class_names[first_label])
  print("Vectorized review", vectorize_text(first_review, first_label))

In [ ]:
# To get word associated with a unique number
vectorize_layer.get_vocabulary()[1287]

In [ ]:
text_col = data['feature_text'].values
target_col = data['target'].values

def custom_standardization(input_data):
  lowercase = tf.strings.lower(input_data)
  stripped_html = tf.strings.regex_replace(lowercase, '<br />', ' ')
  return tf.strings.regex_replace(stripped_html,'[%s]' % re.escape(string.punctuation),'')

vectorize_layer = TextVectorization(
    standardize=custom_standardization,
    max_tokens=None,
    output_mode='int',
    output_sequence_length=None)

vectorize_layer.adapt(text_col) 

model = tf.keras.Sequential([
  vectorize_layer, # Added Vectorize layer here
  Embedding(vocab_size, embedding_dim, name="embedding"),
  tf.keras.layers.LSTM(64),
  tf.keras.layers.Dropout(0.1),
  Dense(1,activation='softplus')
])

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              loss=tf.keras.losses.MeanAbsoluteError(),
              metrics=[tf.keras.metrics.MeanAbsolutePercentageError(), tf.keras.metrics.MeanAbsoluteError()])

In [ ]:
# Extract all unciode characters from text column and then concatenate all characters in differnt rows of a column as a string
df['unicode_chars'] = df['Text_col'].apply(lambda x: re.findall('[^\u0000-\u007e]+',str(x)))
df['unicode_chars'] = [','.join(map(str, l)) for l in df['unicode_chars']]  # concatenate rows as a string
unicode_chars_uq = set(', '.join(df['unicode_chars'].apply(lambda x: str(x))))  # to get distinct characters
unicode_chars_uq
# result = {' ', ',', '\xa0', '®', '°', 'º', '–', '—', '’', '”', '™', '�'}

In [6]:
string_check = "sandy� � ® ° 200gsm (Bright Lycra)"
string_check.encode("utf-8")
# print(string_check.encode('ascii', 'replace'))

b'sandy\xef\xbf\xbd \xef\xbf\xbd \xc2\xae \xc2\xb0 200gsm (Bright Lycra)'

In [3]:
raw_data_sample = 'ത'+'"Pandemonium" -is a horror® movie spoof - !that comes off more stupid than funny.<br /><br />Believe me when I tell you, I love comedies. Especially comedy spoofs. "Airplane", "The Naked Gun" trilogy, "Blazing Saddles", "High Anxiety", and "Spaceballs" are some of my favorite comedies that spoof a particular genre. "Pandemonium" is not up there with those films. Most of the scenes in this movie had me sitting there in stunned silence because the movie wasn\'t all that funny. There are a few laughs in the film, but when you watch a comedy, you expect to laugh a lot more than a few times and that\'s all this film has going for it. Geez, "Scream" had more laughs than this film and that was more of a horror film. How bizarre is that?<br /><br />*1/2 (out of four)'

print(raw_data_sample)
print(('ത'+raw_data_sample).encode("utf-8"))
print(raw_data_sample.encode("utf-8").decode("utf-8"))

# Identify all unicode characters
df['text_col'].apply(lambda x: re.findall('[^\u0000-\u007e]+',str(x)))

string_check = "sandy� � ® ° 200gsm (Bright Lycra)"
print(string_check.encode('ascii', 'replace'))
Df['text_col'].apply(lambda x: x.encode('ascii', 'replace'))
[s.encode('ascii', 'ignore') for s in Df['text_col']]

print(string_check.encode('ascii', 'ignore'))
string_check

# Tensorflow 
# converted � to ? {' ', ',', '\xa0', '®', '°', 'º', '–', '—', '’', '”', '™', '�'}
string_check = tf.constant(u"sandy� � ® ° 200gsm (Bright Lycra)".encode('ascii','ignore'))
print(string_check)

ത"Pandemonium" -is a horror® movie spoof - !that comes off more stupid than funny.<br /><br />Believe me when I tell you, I love comedies. Especially comedy spoofs. "Airplane", "The Naked Gun" trilogy, "Blazing Saddles", "High Anxiety", and "Spaceballs" are some of my favorite comedies that spoof a particular genre. "Pandemonium" is not up there with those films. Most of the scenes in this movie had me sitting there in stunned silence because the movie wasn't all that funny. There are a few laughs in the film, but when you watch a comedy, you expect to laugh a lot more than a few times and that's all this film has going for it. Geez, "Scream" had more laughs than this film and that was more of a horror film. How bizarre is that?<br /><br />*1/2 (out of four)
b'\xe0\xb4\xa4\xe0\xb4\xa4"Pandemonium" -is a horror\xc2\xae movie spoof - !that comes off more stupid than funny.<br /><br />Believe me when I tell you, I love comedies. Especially comedy spoofs. "Airplane", "The Naked Gun" trilog

NameError: ignored

In [ ]:
self.embedding = nn.Embedding(one_hot_dim, embedding_dim)
        self.maxpool1 = nn.MaxPool1d(kernel_size=kernel_size, stride=stride_size, padding=1)
        self.conv1 = nn.Conv1d(embedding_dim, out_channels=embedding_dim, kernel_size=kernel_size, stride=stride_size, padding=1)
        self.maxpool2 = nn.MaxPool1d(kernel_size=kernel_size, stride=stride_size, padding=1)
        self.conv2 = nn.Conv1d(embedding_dim, out_channels=embedding_dim, kernel_size=kernel_size, stride=stride_size, padding=1)
        self.relu = nn.ReLU()
        self.fc1 = nn.Linear(in_features=size_after_conv, out_features=output_dim)

# https://blog.keras.io/using-pre-trained-word-embeddings-in-a-keras-model.html








In [ ]:
re.findall(r'[%s]', raw_data_sample)
print(re.findall(r'[%s] % re.escape(string.punctuation)', raw_data_sample))


[]


In [ ]:
 "résumé".encode("utf-8")

 "El Niño".encode("utf-8")


 b"r\xc3\xa9sum\xc3\xa9".decode("utf-8")
'résumé'
.decode("utf-8")
'El Niño'

In [ ]:
import emoji

def extract_emojis(s):
  return ''.join(c for c in s if c in emoji.UNICODE_EMOJI)

extract_emojis(raw_data_sample)

''

In [ ]:
pattern = re.compile("d")
pattern.search("dog") 

<re.Match object; span=(0, 1), match='d'>